In [76]:
import numpy as np
import pandas as pd 
from tqdm import tqdm

In [123]:
data_champ = pd.read_csv('../data/comptages-routiers-permanents_CHAMPS.csv', sep=';')
data_concen = pd.read_csv('../data/comptages-routiers-permanents-concention.csv', sep=';')
data_sts = pd.read_csv('../data/comptages-routiers-permanents-sts.csv', sep=';')
data_meteo = pd.read_csv('../data/donnees-synop-essentielles-omm.csv', sep=';')
data_vacan = pd.read_csv('../data/vacances_scolaires.csv')

### Vacances data

In [122]:
start_date = pd.to_datetime('2021-12-01')
finish_date = pd.to_datetime('2023-01-27')

data_vacan = data_vacan[['date', 'vacances_zone_c']]
data_vacan['date'] = pd.to_datetime(data_vacan['date'])

data_vacan['vacances_zone_c'].replace(False, 0, inplace=True)
data_vacan['vacances_zone_c'].replace(True, 1, inplace=True)
data_vacan = data_vacan[data_vacan['date'] >= start_date]
data_vacan = data_vacan[data_vacan['date'] <= finish_date]
data_vacan.reset_index(inplace=True)


/var/folders/ty/9ln40ngj6mg6b3f01hcmnfn40000gn/T/ipykernel_85297/740675880.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_vacan['date'] = pd.to_datetime(data_vacan['date'])
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [102]:
data_vacan

,index,date,vacances_zone_c
0,11657,2021-12-01,0
1,11658,2021-12-02,0
2,11659,2021-12-03,0
3,11660,2021-12-04,0
4,11661,2021-12-05,0
...,...,...,...
418,12075,2023-01-23,0
419,12076,2023-01-24,0
420,12077,2023-01-25,0
421,12078,2023-01-26,0


### Champ data && Sts && Concen

In [103]:
data_champ['Date et heure de comptage'] = pd.to_datetime(data_champ['Date et heure de comptage'])
data_concen['Date et heure de comptage'] = pd.to_datetime(data_concen['Date et heure de comptage'])
data_sts['Date et heure de comptage'] = pd.to_datetime(data_sts['Date et heure de comptage'])

data_champ = data_champ[data_champ['Libelle noeud amont'] == 'Av_Champs_Elysees-Washington']
data_champ = data_champ[data_champ['Libelle noeud aval'] == 'Av_Champs_Elysees-Berri']

data_concen = data_concen[data_concen['Libelle noeud amont'] == 'Lecourbe-Convention']
data_concen = data_concen[data_concen['Libelle noeud aval'] == 'Convention-Blomet']

data_sts = data_sts[data_sts['Libelle noeud amont'] == 'Sts_Peres-Voltaire']
data_sts = data_sts[data_sts['Libelle noeud aval'] == 'Sts_Peres-Universite']


data_streets = data_champ.append([data_sts, data_concen]).sort_values(by='Date et heure de comptage')

In [104]:
target = data_streets[['Débit horaire', 'Taux d\'occupation']]

indexes_to_drop = ['Identifiant arc', 'Identifiant noeud amont', 'Libelle noeud amont', 'Identifiant noeud aval', 'Libelle noeud aval', 
                   'Date debut dispo data', 'Date fin dispo data', 'geo_point_2d', 'geo_shape', 'Débit horaire', 'Taux d\'occupation']

data_streets.drop(indexes_to_drop, axis=1, inplace=True)

In [105]:
data_streets

,Libelle,Date et heure de comptage,Etat trafic,Etat arc
636,AV_Champs_Elysees,2021-12-01 04:00:00+01:00,Pré-saturé,Invalide
15017,Sts_Peres,2021-12-01 04:00:00+01:00,Fluide,Invalide
2534,Convention,2021-12-01 04:00:00+01:00,Fluide,Invalide
15713,AV_Champs_Elysees,2021-12-01 05:00:00+01:00,Fluide,Invalide
378,Sts_Peres,2021-12-01 05:00:00+01:00,Fluide,Invalide
...,...,...,...,...
179962,Convention,2023-01-27 20:00:00+01:00,Fluide,Invalide
179960,Convention,2023-01-27 21:00:00+01:00,Fluide,Invalide
179186,Convention,2023-01-27 22:00:00+01:00,Fluide,Invalide
179185,Convention,2023-01-27 23:00:00+01:00,Fluide,Invalide


### Meteo

In [124]:
indexes_meteo = ['Date', 'Variation de pression en 3 heures', 'Direction du vent moyen 10 mn', 
                'Vitesse du vent moyen 10 mn', 'Température', 'Humidité', 'Visibilité horizontale', 'Temps présent',
                'Temps passé 1']

discription = {'pluie' : 1, 'neige': 2}

def weather(data, discription):
    temps_present = data['Temps présent'].to_list()
    temps_pass = data['Temps passé 1'].to_list()
    
    curr_weather = []
    pass_weather = []
    
    for i in range(len(temps_pass)):
        curr_1, curr_2 = 0, 0
        s1, s2 = temps_present[i], temps_pass[i]
        
        
        for k in discription.keys():
            
            if k in str(s1):
                curr_1 += discription[k]
                
            if k in str(s2):
                curr_2 += discription[k]
                
        curr_weather.append(curr_1)
        pass_weather.append(curr_2)
        
    return (curr_weather, pass_weather)

In [125]:
data_meteo = data_meteo[indexes_meteo]
data_meteo['Date'] = pd.to_datetime(data_meteo['Date'])
data_meteo.sort_values('Date', inplace=True)
data_meteo['Meteo pres'], data_meteo['Meteo pass'] = weather(data_meteo, discription) 
data_meteo.drop(['Temps présent', 'Temps passé 1'], axis=1, inplace=True)


data_meteo

,Date,Variation de pression en 3 heures,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Humidité,Visibilité horizontale,Meteo pres,Meteo pass
2272,2021-01-01 01:00:00+01:00,80.0,270,1.8,272.75,96,990.0,0,0
222,2021-01-01 04:00:00+01:00,110.0,300,1.7,271.25,98,210.0,0,0
3528,2021-01-01 07:00:00+01:00,30.0,290,2.6,271.95,98,3660.0,0,0
3901,2021-01-01 10:00:00+01:00,150.0,280,1.7,272.45,97,3500.0,0,0
3529,2021-01-01 13:00:00+01:00,30.0,50,1.0,276.95,82,8000.0,0,0
...,...,...,...,...,...,...,...,...,...
2794,2023-01-27 07:00:00+01:00,-30.0,10,5.2,277.85,78,15000.0,0,0
182,2023-01-27 10:00:00+01:00,80.0,40,7.3,277.95,71,20000.0,0,0
1230,2023-01-27 13:00:00+01:00,10.0,30,5.9,277.75,76,20000.0,0,0
3160,2023-01-27 16:00:00+01:00,40.0,40,5.6,277.55,74,22570.0,0,0


In [126]:
start_date_meteo = pd.to_datetime('2021-12-01 04:00:00+01:00')
finish_date_meteo = pd.to_datetime('2023-01-28 00:00:00+01:00')

data_meteo = data_meteo[data_meteo['Date'] >= start_date_meteo]
data_meteo = data_meteo[data_meteo['Date'] <= finish_date_meteo]
data_meteo.reset_index(inplace=True)

In [127]:
data_meteo

,index,Date,Variation de pression en 3 heures,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Humidité,Visibilité horizontale,Meteo pres,Meteo pass
0,18,2021-12-01 04:00:00+01:00,-420.0,230,7.3,281.45,90,15000.0,0,0
1,4453,2021-12-01 07:00:00+01:00,-460.0,210,7.9,281.55,97,10000.0,0,0
2,5153,2021-12-01 10:00:00+01:00,-160.0,240,6.3,282.95,93,25000.0,0,0
3,4712,2021-12-01 13:00:00+01:00,10.0,270,8.3,284.15,77,25000.0,0,0
4,3360,2021-12-01 16:00:00+01:00,50.0,290,6.4,282.35,75,25000.0,1,0
...,...,...,...,...,...,...,...,...,...,...
3375,2794,2023-01-27 07:00:00+01:00,-30.0,10,5.2,277.85,78,15000.0,0,0
3376,182,2023-01-27 10:00:00+01:00,80.0,40,7.3,277.95,71,20000.0,0,0
3377,1230,2023-01-27 13:00:00+01:00,10.0,30,5.9,277.75,76,20000.0,0,0
3378,3160,2023-01-27 16:00:00+01:00,40.0,40,5.6,277.55,74,22570.0,0,0


### Unification of the data 

In [110]:
data = data_streets
data.reset_index(inplace=True)
data['Vacance'] = 0
data

,index,Libelle,Date et heure de comptage,Etat trafic,Etat arc,Vacance
0,636,AV_Champs_Elysees,2021-12-01 04:00:00+01:00,Pré-saturé,Invalide,0
1,15017,Sts_Peres,2021-12-01 04:00:00+01:00,Fluide,Invalide,0
2,2534,Convention,2021-12-01 04:00:00+01:00,Fluide,Invalide,0
3,15713,AV_Champs_Elysees,2021-12-01 05:00:00+01:00,Fluide,Invalide,0
4,378,Sts_Peres,2021-12-01 05:00:00+01:00,Fluide,Invalide,0
...,...,...,...,...,...,...
29223,179962,Convention,2023-01-27 20:00:00+01:00,Fluide,Invalide,0
29224,179960,Convention,2023-01-27 21:00:00+01:00,Fluide,Invalide,0
29225,179186,Convention,2023-01-27 22:00:00+01:00,Fluide,Invalide,0
29226,179185,Convention,2023-01-27 23:00:00+01:00,Fluide,Invalide,0


In [72]:
data['Date et heure de comptage'][0].date() == data_vacan['date'][0].date()

True

In [78]:
def add_vacance(data_vacan, data):
    
    for j in tqdm(range(data_vacan.shape[0])):
        for i in range(data.shape[0]):
            if data['Date et heure de comptage'][i].date() == data_vacan['date'][j].date() :
                data['Vacance'][i] = data_vacan['vacances_zone_c'][j]
    return data
data = add_vacance(data_vacan, data)

  0%|                                                                    | 0/423 [00:00<?, ?it/s]/var/folders/ty/9ln40ngj6mg6b3f01hcmnfn40000gn/T/ipykernel_85297/3160164772.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Vacance'][i] = data_vacan['vacances_zone_c'][j]
100%|██████████████████████████████████████████████████████████| 423/423 [02:46<00:00,  2.54it/s]


In [81]:
data

,index,Libelle,Date et heure de comptage,Etat trafic,Etat arc,Vacance
0,636,AV_Champs_Elysees,2021-12-01 04:00:00+01:00,Pré-saturé,Invalide,0
1,15017,Sts_Peres,2021-12-01 04:00:00+01:00,Fluide,Invalide,0
2,2534,Convention,2021-12-01 04:00:00+01:00,Fluide,Invalide,0
3,15713,AV_Champs_Elysees,2021-12-01 05:00:00+01:00,Fluide,Invalide,0
4,378,Sts_Peres,2021-12-01 05:00:00+01:00,Fluide,Invalide,0
...,...,...,...,...,...,...
29223,179962,Convention,2023-01-27 20:00:00+01:00,Fluide,Invalide,0
29224,179960,Convention,2023-01-27 21:00:00+01:00,Fluide,Invalide,0
29225,179186,Convention,2023-01-27 22:00:00+01:00,Fluide,Invalide,0
29226,179185,Convention,2023-01-27 23:00:00+01:00,Fluide,Invalide,0


In [114]:
data_meteo.iloc[0:7]

,index,Date,Variation de pression en 3 heures,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Humidité,Visibilité horizontale,Meteo pres,Meteo pass
2670,18,2021-12-01 04:00:00+01:00,-420.0,230,7.3,281.45,90,15000.0,0,0
2671,4453,2021-12-01 07:00:00+01:00,-460.0,210,7.9,281.55,97,10000.0,0,0
2672,5153,2021-12-01 10:00:00+01:00,-160.0,240,6.3,282.95,93,25000.0,0,0
2673,4712,2021-12-01 13:00:00+01:00,10.0,270,8.3,284.15,77,25000.0,0,0
2674,3360,2021-12-01 16:00:00+01:00,50.0,290,6.4,282.35,75,25000.0,1,0
2675,1285,2021-12-01 19:00:00+01:00,120.0,260,5.2,280.35,81,25000.0,0,0
2676,3773,2021-12-01 22:00:00+01:00,80.0,270,4.3,279.15,87,31490.0,0,0


In [233]:
data_meteo.tail(10)

,index,Date,Variation de pression en 3 heures,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Humidité,Visibilité horizontale,Meteo pres,Meteo pass
3370,2793,2023-01-26 16:00:00+01:00,-50.0,30,5.3,276.25,94,10000.0,0,0
3371,902,2023-01-26 19:00:00+01:00,80.0,20,5.8,278.15,89,14000.0,2,0
3372,3885,2023-01-26 22:00:00+01:00,0.0,360,4.5,277.85,84,15000.0,0,0
3373,2791,2023-01-27 01:00:00+01:00,-70.0,10,5.7,278.05,83,24510.0,0,0
3374,1802,2023-01-27 04:00:00+01:00,-60.0,10,4.9,277.95,79,15000.0,0,0
3375,2794,2023-01-27 07:00:00+01:00,-30.0,10,5.2,277.85,78,15000.0,0,0
3376,182,2023-01-27 10:00:00+01:00,80.0,40,7.3,277.95,71,20000.0,0,0
3377,1230,2023-01-27 13:00:00+01:00,10.0,30,5.9,277.75,76,20000.0,0,0
3378,3160,2023-01-27 16:00:00+01:00,40.0,40,5.6,277.55,74,22570.0,0,0
3379,1365,2023-01-27 19:00:00+01:00,160.0,30,5.9,277.25,78,11870.0,0,0


In [133]:
data_meteo['Date'][0] + pd.DateOffset(hours=1)

Timestamp('2021-12-01 05:00:00+0100', tz='tzoffset(None, 3600)')

In [166]:
def df_meteo(data_meteo):
    
    h = pd.DateOffset(hours=1)
    
    
    for i in tqdm(range(data_meteo.shape[0] - 1)):
        df = data_meteo.iloc[i]
        
        while df['Date'] + h < data_meteo['Date'][i + 1]:
            df['Date'] += h
            data_meteo = data_meteo.append(df)
    
    return data_meteo

temp = df_meteo(data_meteo).sort_values(by="Date")

In [176]:
temp.reset_index(inplace=True)

In [229]:
columns_meteo = data_meteo.columns.to_list()
columns_meteo.remove('Date')
columns_meteo.remove('index')
columns_meteo



['Variation de pression en 3 heures',
 'Direction du vent moyen 10 mn',
 'Vitesse du vent moyen 10 mn',
 'Température',
 'Humidité',
 'Visibilité horizontale',
 'Meteo pres',
 'Meteo pass']

In [219]:
data[columns_meteo].iloc[0] = [2, 2, 2, 2, 2, 2, 2, 2]
data[columns_meteo[0]][0] = 2
data

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/var/folders/ty/9ln40ngj6mg6b3f01hcmnfn40000gn/T/ipykernel_85297/1711813117.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

,index,Libelle,Date et heure de comptage,Etat trafic,Etat arc,Vacance,Variation de pression en 3 heures,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Humidité,Visibilité horizontale,Meteo pres,Meteo pass
0,636,AV_Champs_Elysees,2021-12-01 04:00:00+01:00,Pré-saturé,Invalide,0,2,0,0,0,0,0,0,0
1,15017,Sts_Peres,2021-12-01 04:00:00+01:00,Fluide,Invalide,0,0,0,0,0,0,0,0,0
2,2534,Convention,2021-12-01 04:00:00+01:00,Fluide,Invalide,0,0,0,0,0,0,0,0,0
3,15713,AV_Champs_Elysees,2021-12-01 05:00:00+01:00,Fluide,Invalide,0,0,0,0,0,0,0,0,0
4,378,Sts_Peres,2021-12-01 05:00:00+01:00,Fluide,Invalide,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223,179962,Convention,2023-01-27 20:00:00+01:00,Fluide,Invalide,0,0,0,0,0,0,0,0,0
29224,179960,Convention,2023-01-27 21:00:00+01:00,Fluide,Invalide,0,0,0,0,0,0,0,0,0
29225,179186,Convention,2023-01-27 22:00:00+01:00,Fluide,Invalide,0,0,0,0,0,0,0,0,0
29226,179185,Convention,2023-01-27 23:00:00+01:00,Fluide,Invalide,0,0,0,0,0,0,0,0,0


In [224]:
for i in range(0, 5):
    print(i)

0
1
2
3
4


In [225]:
def dfs_injection(meteo, data, new_columns, num_streets=3):
    for i in tqdm(range(meteo.shape[0])):
        counter = 0
        for j in range(i, data.shape[0]):
            if counter > num_streets:
                break
            if data['Date et heure de comptage'][j] == meteo['Date'][i]:
                counter += 1
                for col in new_columns:
                    data[col][j] = meteo[col][i]
    return data

In [226]:
data = dfs_injection(temp, data, columns_meteo)

  0%|                                                                  | 0/10144 [00:00<?, ?it/s]/var/folders/ty/9ln40ngj6mg6b3f01hcmnfn40000gn/T/ipykernel_85297/2069170027.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col][j] = meteo[col][i]
100%|██████████████████████████████████████████████████████| 10144/10144 [39:37<00:00,  4.27it/s]


In [242]:
data.to_csv('almost_data.csv')

In [235]:
data.head(10)

,index,Libelle,Date et heure de comptage,Etat trafic,Etat arc,Vacance,Variation de pression en 3 heures,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Humidité,Visibilité horizontale,Meteo pres,Meteo pass
0,636,AV_Champs_Elysees,2021-12-01 04:00:00+01:00,Pré-saturé,Invalide,0,-420,230,7,281,90,15000,0,0
1,15017,Sts_Peres,2021-12-01 04:00:00+01:00,Fluide,Invalide,0,-420,230,7,281,90,15000,0,0
2,2534,Convention,2021-12-01 04:00:00+01:00,Fluide,Invalide,0,-420,230,7,281,90,15000,0,0
3,15713,AV_Champs_Elysees,2021-12-01 05:00:00+01:00,Fluide,Invalide,0,-420,230,7,281,90,15000,0,0
4,378,Sts_Peres,2021-12-01 05:00:00+01:00,Fluide,Invalide,0,-420,230,7,281,90,15000,0,0
5,2532,Convention,2021-12-01 05:00:00+01:00,Fluide,Invalide,0,-420,230,7,281,90,15000,0,0
6,635,AV_Champs_Elysees,2021-12-01 06:00:00+01:00,Fluide,Invalide,0,-420,230,7,281,90,15000,0,0
7,15016,Sts_Peres,2021-12-01 06:00:00+01:00,Fluide,Invalide,0,-420,230,7,281,90,15000,0,0
8,80537,Convention,2021-12-01 06:00:00+01:00,Fluide,Invalide,0,-420,230,7,281,90,15000,0,0
9,639,AV_Champs_Elysees,2021-12-01 07:00:00+01:00,Fluide,Invalide,0,-460,210,7,281,97,10000,0,0


### Scaling 

In [243]:
cat_columns = ['Libelle', 'Etat trafic', 'Etat arc']

In [244]:
pd.get_dummies(data[cat_columns])

,Libelle_AV_Champs_Elysees,Libelle_Convention,Libelle_Sts_Peres,Etat trafic_Bloqué,Etat trafic_Fluide,Etat trafic_Inconnu,Etat trafic_Pré-saturé,Etat trafic_Saturé,Etat arc_Barré,Etat arc_Invalide
0,1,0,0,0,0,0,1,0,0,1
1,0,0,1,0,1,0,0,0,0,1
2,0,1,0,0,1,0,0,0,0,1
3,1,0,0,0,1,0,0,0,0,1
4,0,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
29223,0,1,0,0,1,0,0,0,0,1
29224,0,1,0,0,1,0,0,0,0,1
29225,0,1,0,0,1,0,0,0,0,1
29226,0,1,0,0,1,0,0,0,0,1


In [248]:
data = pd.concat([data, pd.get_dummies(data[cat_columns])], axis=1).drop(cat_columns, axis=1)

In [276]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [268]:
data.to_csv('almost_data.csv')

In [269]:
data = pd.read_csv('almost_data.csv')

In [274]:
data.set_index('Date et heure de comptage', inplace=True)


In [277]:
data

,Vacance,Variation de pression en 3 heures,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Humidité,Visibilité horizontale,Meteo pres,Meteo pass,Libelle_AV_Champs_Elysees,Libelle_Convention,Libelle_Sts_Peres,Etat trafic_Bloqué,Etat trafic_Fluide,Etat trafic_Inconnu,Etat trafic_Pré-saturé,Etat trafic_Saturé,Etat arc_Barré,Etat arc_Invalide
Date et heure de comptage,,,,,,,,,,,,,,,,,,,
2021-12-01 04:00:00+01:00,0,-420,230,7,281,90,15000,0,0,1,0,0,0,0,0,1,0,0,1
2021-12-01 04:00:00+01:00,0,-420,230,7,281,90,15000,0,0,0,0,1,0,1,0,0,0,0,1
2021-12-01 04:00:00+01:00,0,-420,230,7,281,90,15000,0,0,0,1,0,0,1,0,0,0,0,1
2021-12-01 05:00:00+01:00,0,-420,230,7,281,90,15000,0,0,1,0,0,0,1,0,0,0,0,1
2021-12-01 05:00:00+01:00,0,-420,230,7,281,90,15000,0,0,0,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-27 20:00:00+01:00,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
2023-01-27 21:00:00+01:00,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
2023-01-27 22:00:00+01:00,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1


In [278]:
data = pd.read_csv('almost_data.csv')

In [279]:
data

,Unnamed: 0,Date et heure de comptage,Vacance,Variation de pression en 3 heures,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Humidité,Visibilité horizontale,Meteo pres,...,Libelle_AV_Champs_Elysees,Libelle_Convention,Libelle_Sts_Peres,Etat trafic_Bloqué,Etat trafic_Fluide,Etat trafic_Inconnu,Etat trafic_Pré-saturé,Etat trafic_Saturé,Etat arc_Barré,Etat arc_Invalide
0,0,2021-12-01 04:00:00+01:00,0,-420,230,7,281,90,15000,0,...,1,0,0,0,0,0,1,0,0,1
1,1,2021-12-01 04:00:00+01:00,0,-420,230,7,281,90,15000,0,...,0,0,1,0,1,0,0,0,0,1
2,2,2021-12-01 04:00:00+01:00,0,-420,230,7,281,90,15000,0,...,0,1,0,0,1,0,0,0,0,1
3,3,2021-12-01 05:00:00+01:00,0,-420,230,7,281,90,15000,0,...,1,0,0,0,1,0,0,0,0,1
4,4,2021-12-01 05:00:00+01:00,0,-420,230,7,281,90,15000,0,...,0,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223,29223,2023-01-27 20:00:00+01:00,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
29224,29224,2023-01-27 21:00:00+01:00,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
29225,29225,2023-01-27 22:00:00+01:00,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
29226,29226,2023-01-27 23:00:00+01:00,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
